<a href="https://colab.research.google.com/github/Zenith1618/Web-Scraping/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date, timedelta

def get_valid_dates(year):
    start_date = date(year, 1, 1)
    end_date = date(year, 12, 31)
    delta = timedelta(days=1)
    valid_dates = []

    current_date = start_date
    while current_date <= end_date:
        valid_dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += delta

    return valid_dates

year = 2021  # Change this to the desired year
valid_dates = get_valid_dates(year)
print(valid_dates)


['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021

In [ ]:
!pip install MechanicalSoup
!pip install PyPDF2
!pip install tabula-py pandas openpyxl

import requests
import pandas as pd
import mechanicalsoup
from bs4 import BeautifulSoup
import tabula
from urllib.parse import urljoin
import PyPDF2

browser = mechanicalsoup.Browser()

url = 'https://report.grid-india.in/psp_report.php'
page = browser.get(url, verify=False)





/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
first_month_dates = [d for d in valid_dates if d.startswith(f"{year}-01")]
print(first_month_dates)

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31']


In [ ]:
data_list = []

for input_date in first_month_dates:
  # input_date = "2022-08-14"  # Replace with the desired date
  form = page.soup.find("form", class_="date-form")
  date_input = form.find("input", type="date")
  date_input["value"] = input_date

  response = browser.submit(form, page.url, verify=False)

  #print(response.text)


  soup = BeautifulSoup(response.text, "html.parser")
  pdf_links = soup.find_all("a", href=True)
  pdf_link = None

  for link in pdf_links:
      if link["href"].endswith(".pdf"):
          pdf_link = link["href"]
          print("Found PDF link:", pdf_link)
          break

  if not pdf_link:
      print("No PDF link found.")
      exit()



  # Step 6: Download the PDF file
  base_link = "https://report.grid-india.in/"
  down_link = urljoin(base_link, pdf_link)
  pdf_response = requests.get(down_link, verify=False)
  pdf_filename = "temp.pdf"
  with open(pdf_filename, "wb") as pdf_file:
      pdf_file.write(pdf_response.content)




  #Use PyPDF2 to extract content from page 2
  pdf_filename = "temp.pdf"  # Replace with the actual filename
  output_pdf_filename = "page2.pdf"  # Replace with the desired output filename

  with open(pdf_filename, "rb") as pdf_file:
      pdf_reader = PyPDF2.PdfReader(pdf_file)
      pdf_writer = PyPDF2.PdfWriter()

      # Add only page 2 to the new PDF writer
      pdf_writer.add_page(pdf_reader.pages[1])

      # Save the new PDF containing only page 2
      with open(output_pdf_filename, "wb") as output_pdf_file:
          pdf_writer.write(output_pdf_file)

  print("Page 2 extracted and saved to", output_pdf_filename)





  pdf_path = "temp.pdf"

  tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)

  # Assume you want the first table (use indexing if there are multiple tables)
  table_data = tables[0]



  # Convert the table data into a pandas DataFrame
  df = pd.DataFrame(table_data)

  # Save the DataFrame into an Excel file
  excel_output = "output.xlsx"
  df.to_excel(excel_output, index=False, engine="openpyxl")
  print("Data extracted and saved to", excel_output)

  # Load the output Excel file
  output_excel_file = "output.xlsx"  # Replace with the actual filename
  df_output = pd.read_excel(output_excel_file)

  # Extract the date and total power columns
  date = input_date  # Assuming the date is in the first row, first column
  total_power = df_output.iloc[0, 6]  # Assuming total power is in the second row, seventh column

  data_list.append({"Date": input_date, "Total Power": total_power})


new_df = pd.DataFrame(data_list)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found PDF link: ./ReportData/Daily Report/PSP Report/2020-2021/January 2021/01.01.21_NLDC_PSP.pdf


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Page 2 extracted and saved to page2.pdf
Data extracted and saved to output.xlsx


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Data extracted and saved to output.xlsx
Found PDF link: ./ReportData/Daily Report/PSP Report/2020-2021/January 2021/31.01.21_NLDC_PSP.pdf


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Page 2 extracted and saved to page2.pdf


Aug 15, 2023 8:27:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 8:27:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 8:27:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Data extracted and saved to output.xlsx


In [ ]:
new_df.head()

,Date,Total Power
0,2021-01-01,165169
1,2021-01-02,158762
2,2021-01-03,162253
3,2021-01-04,157820
4,2021-01-05,158938


In [ ]:
new_df.to_excel('jan.xlsx', index=False, header=True)